In [1]:
!pwd

/home/jovyan/work/notebooks


In [2]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/
                        
Using Scala version 2.11.12, OpenJDK 64-Bit Server VM, 1.8.0_252
Branch HEAD
Compiled by user centos on 2020-02-02T19:38:06Z
Revision cee4ecbb16917fa85f02c635925e2687400aa56b
Url https://gitbox.apache.org/repos/asf/spark.git
Type --help for more information.


In [3]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('wiki-changes-event-consumer')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
         .getOrCreate())
sc = spark.sparkContext

In [4]:
# Create stream dataframe setting kafka server, topic and offset option
df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka-server:9092") # kafka server
  .option("subscribe", "wiki-changes") # topic
  #.option("startingOffsets", "earliest") # start from beginning 
  .load())

In [5]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
# Start query stream over stream dataframe
queryStreamMem =(
    df
    .writeStream
    .format("memory")
    .queryName("wiki_changes")
    .outputMode("append")
    .start())

In [7]:
print(queryStreamMem.lastProgress)

None


In [8]:
if queryStreamMem.lastProgress:
    print(queryStreamMem.lastProgress)

In [9]:
from pyspark.sql.functions import from_json, col, from_unixtime, to_date, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, LongType, IntegerType
from time import sleep
from IPython.display import clear_output

# Event data schema
schema_wiki = StructType([StructField("$schema",StringType(),True),StructField("bot",BooleanType(),True),StructField("comment",StringType(),True),StructField("id",StringType(),True),StructField("length",StructType([StructField("new",IntegerType(),True),StructField("old",IntegerType(),True)]),True),StructField("meta",StructType([StructField("domain",StringType(),True),StructField("dt",StringType(),True),StructField("id",StringType(),True),StructField("offset",LongType(),True),StructField("partition",LongType(),True),StructField("request_id",StringType(),True),StructField("stream",StringType(),True),StructField("topic",StringType(),True),StructField("uri",StringType(),True)]),True),StructField("minor",BooleanType(),True),StructField("namespace",IntegerType(),True),StructField("parsedcomment",StringType(),True),StructField("patrolled",BooleanType(),True),StructField("revision",StructType([StructField("new",IntegerType(),True),StructField("old",IntegerType(),True)]),True),StructField("server_name",StringType(),True),StructField("server_script_path",StringType(),True),StructField("server_url",StringType(),True),StructField("timestamp",StringType(),True),StructField("title",StringType(),True),StructField("type",StringType(),True),StructField("user",StringType(),True),StructField("wiki",StringType(),True)])

try:
    i=1
    # While stream is active, fill parquet files 
    while len(spark.streams.active) > 0:
        # Clear output
        clear_output(wait=True)
        print("Run:{}".format(i))
        if queryStreamMem.lastProgress:
            print("Input Rows:{}".format(queryStreamMem.lastProgress["numInputRows"]))
            print("Input Rows per second:{}".format(queryStreamMem.lastProgress["inputRowsPerSecond"]))

        # Count number of events
        spark.sql("select count(1) as qty from wiki_changes").show()
        
        # Convert binary to string
        df_kafka =  spark.sql("select CAST(key as string) key, CAST(value as string) value, topic, timestamp from wiki_changes")
        
        # Create dataframe setting schema for event data
        df_wiki = (df_kafka
                   # Sets schema for event data
                   .withColumn("value", from_json("value", schema_wiki))
                  )

        # Transform into tabular and convert unix timestamp to timestamp
        df_wiki_formatted = (df_wiki.select(
            col("key").alias("event_key")
            ,col("topic").alias("event_topic")
            ,col("timestamp").alias("event_timestamp")
            ,col("value.$schema").alias("schema")
            ,"value.bot"
            ,"value.comment"
            ,"value.id"
            ,col("value.length.new").alias("length_new")
            ,col("value.length.old").alias("length_old")
            ,"value.minor"
            ,"value.namespace"
            ,"value.parsedcomment"
            ,"value.patrolled"
            ,col("value.revision.new").alias("revision_new")
            ,col("value.revision.old").alias("revision_old")
            ,"value.server_name"
            ,"value.server_script_path"
            ,"value.server_url"
            ,to_timestamp(from_unixtime(col("value.timestamp"))).alias("change_timestamp")
            ,to_date(from_unixtime(col("value.timestamp"))).alias("change_timestamp_date")
            ,"value.title"
            ,"value.type"
            ,"value.user"
            ,"value.wiki"
            ,col("value.meta.domain").alias("meta_domain")
            ,col("value.meta.dt").alias("meta_dt")
            ,col("value.meta.id").alias("meta_id")
            ,col("value.meta.offset").alias("meta_offset")
            ,col("value.meta.partition").alias("meta_partition")
            ,col("value.meta.request_id").alias("meta_request_id")
            ,col("value.meta.stream").alias("meta_stream")
            ,col("value.meta.topic").alias("meta_topic")
            ,col("value.meta.uri").alias("meta_uri")
        ))
        
        # Write to data lake parquet partitioned
        df_wiki_formatted.write.mode('append').partitionBy("change_timestamp_date", "server_name").parquet("/home/jovyan/work/data-lake/wiki-changes")
        
        sleep(5)
        i=i+1
        
except KeyboardInterrupt:
    # Stop Query Stream
    queryStreamMem.stop()
    
    print("stream process interrupted")

Run:150
Input Rows:0
Input Rows per second:0.0
+-----+
|  qty|
+-----+
|17555|
+-----+

stream process interrupted


### Auxiliar commands

In [7]:
# Check active streams
for s in spark.streams.active:
    print("ID:{} | NAME:{}".format(s.id, s.name))

In [13]:
# Count number of events
spark.sql("select count(1) from wiki_changes").show()

+--------+
|count(1)|
+--------+
|   12316|
+--------+



In [104]:
# Stop Query Stream
queryStreamMem.stop()

In [73]:
# Convert binary to string
df_kafka =  spark.sql("select CAST(key as string) key, CAST(value as string) value, topic, timestamp from wiki_changes")

In [74]:
df_kafka.show()

+----+--------------------+------------+--------------------+
| key|               value|       topic|           timestamp|
+----+--------------------+------------+--------------------+
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"$schema": "/med...|wiki-changes|2020-07-19 07:42:...|
|null|{"

In [75]:
# Event data schema
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, LongType, IntegerType
schema_wiki = StructType([StructField("$schema",StringType(),True),StructField("bot",BooleanType(),True),StructField("comment",StringType(),True),StructField("id",StringType(),True),StructField("length",StructType([StructField("new",IntegerType(),True),StructField("old",IntegerType(),True)]),True),StructField("meta",StructType([StructField("domain",StringType(),True),StructField("dt",StringType(),True),StructField("id",StringType(),True),StructField("offset",LongType(),True),StructField("partition",LongType(),True),StructField("request_id",StringType(),True),StructField("stream",StringType(),True),StructField("topic",StringType(),True),StructField("uri",StringType(),True)]),True),StructField("minor",BooleanType(),True),StructField("namespace",IntegerType(),True),StructField("parsedcomment",StringType(),True),StructField("patrolled",BooleanType(),True),StructField("revision",StructType([StructField("new",IntegerType(),True),StructField("old",IntegerType(),True)]),True),StructField("server_name",StringType(),True),StructField("server_script_path",StringType(),True),StructField("server_url",StringType(),True),StructField("timestamp",StringType(),True),StructField("title",StringType(),True),StructField("type",StringType(),True),StructField("user",StringType(),True),StructField("wiki",StringType(),True)])

In [76]:
# Create dataframe setting schema for event data
from pyspark.sql.functions import from_json
df_wiki = (df_kafka
           # Sets schema for event data
           .withColumn("value", from_json("value", schema_wiki))
          )

In [77]:
df_wiki.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- $schema: string (nullable = true)
 |    |-- bot: boolean (nullable = true)
 |    |-- comment: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- length: struct (nullable = true)
 |    |    |-- new: integer (nullable = true)
 |    |    |-- old: integer (nullable = true)
 |    |-- meta: struct (nullable = true)
 |    |    |-- domain: string (nullable = true)
 |    |    |-- dt: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- offset: long (nullable = true)
 |    |    |-- partition: long (nullable = true)
 |    |    |-- request_id: string (nullable = true)
 |    |    |-- stream: string (nullable = true)
 |    |    |-- topic: string (nullable = true)
 |    |    |-- uri: string (nullable = true)
 |    |-- minor: boolean (nullable = true)
 |    |-- namespace: integer (nullable = true)
 |    |-- parsedcomment: string (nullable = true)
 |    |-- patrol

In [78]:
# Transform into tabular and convert unix timestamp to timestamp
from pyspark.sql.functions import col, from_unixtime, to_date, to_timestamp

df_wiki_formatted = (df_wiki.select(
    col("key").alias("event_key")
    ,col("topic").alias("event_topic")
    ,col("timestamp").alias("event_timestamp")
    ,col("value.$schema").alias("schema")
    ,"value.bot"
    ,"value.comment"
    ,"value.id"
    ,col("value.length.new").alias("length_new")
    ,col("value.length.old").alias("length_old")
    ,"value.minor"
    ,"value.namespace"
    ,"value.parsedcomment"
    ,"value.patrolled"
    ,col("value.revision.new").alias("revision_new")
    ,col("value.revision.old").alias("revision_old")
    ,"value.server_name"
    ,"value.server_script_path"
    ,"value.server_url"
    ,to_timestamp(from_unixtime(col("value.timestamp"))).alias("change_timestamp")
    ,to_date(from_unixtime(col("value.timestamp"))).alias("change_timestamp_date")
    ,"value.title"
    ,"value.type"
    ,"value.user"
    ,"value.wiki"
    ,col("value.meta.domain").alias("meta_domain")
    ,col("value.meta.dt").alias("meta_dt")
    ,col("value.meta.id").alias("meta_id")
    ,col("value.meta.offset").alias("meta_offset")
    ,col("value.meta.partition").alias("meta_partition")
    ,col("value.meta.request_id").alias("meta_request_id")
    ,col("value.meta.stream").alias("meta_stream")
    ,col("value.meta.topic").alias("meta_topic")
    ,col("value.meta.uri").alias("meta_uri")
))

In [79]:
df_wiki_formatted.printSchema()

root
 |-- event_key: string (nullable = true)
 |-- event_topic: string (nullable = true)
 |-- event_timestamp: timestamp (nullable = true)
 |-- schema: string (nullable = true)
 |-- bot: boolean (nullable = true)
 |-- comment: string (nullable = true)
 |-- id: string (nullable = true)
 |-- length_new: integer (nullable = true)
 |-- length_old: integer (nullable = true)
 |-- minor: boolean (nullable = true)
 |-- namespace: integer (nullable = true)
 |-- parsedcomment: string (nullable = true)
 |-- patrolled: boolean (nullable = true)
 |-- revision_new: integer (nullable = true)
 |-- revision_old: integer (nullable = true)
 |-- server_name: string (nullable = true)
 |-- server_script_path: string (nullable = true)
 |-- server_url: string (nullable = true)
 |-- change_timestamp: timestamp (nullable = true)
 |-- change_timestamp_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- user: string (nullable = true)
 |-- wiki: string (nullab

In [80]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 500)

df_wiki_formatted.toPandas()

,event_key,event_topic,event_timestamp,schema,bot,comment,id,length_new,length_old,minor,namespace,parsedcomment,patrolled,revision_new,revision_old,server_name,server_script_path,server_url,change_timestamp,change_timestamp_date,title,type,user,wiki,meta_domain,meta_dt,meta_id,meta_offset,meta_partition,meta_request_id,meta_stream,meta_topic,meta_uri
0,None,wiki-changes,2020-07-19 07:42:36.192,/mediawiki/recentchange/1.0.0,False,,106903692,4707.0,4709.0,False,0,,None,2.898080e+07,2.455228e+07,uk.wikipedia.org,/w,https://uk.wikipedia.org,2020-07-19 07:42:35,2020-07-19,Ткачонак Олександр Леонідович,edit,Vity OKM,ukwiki,uk.wikipedia.org,2020-07-19T07:42:35Z,d6e1e2d3-f9e8-4694-a4e2-aa7f6b5e24b6,2574275209,0,972f572a-9f44-4e71-87c9-d581d0a92d2b,mediawiki.recentchange,eqiad.mediawiki.recentchange,https://uk.wikipedia.org/wiki/%D0%A2%D0%BA%D0%B0%D1%87%D0%BE%D0%BD%D0%B0%D0%BA_%D0%9E%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_%D0%9B%D0%B5%D0%BE%D0%BD%D1%96%D0%B4%D0%BE%D0%B2%D0%B8%D1%87
1,None,wiki-changes,2020-07-19 07:42:36.220,/mediawiki/recentchange/1.0.0,False,"Gemaakt door het vertalen van de pagina ""[[:en:Special:Redirect/revision/958575714|Musée Mécanique]]""",113373949,4766.0,0.0,False,2,"Gemaakt door het vertalen van de pagina &quot;<a href=""https://en.wikipedia.org/wiki/Special:Redirect/revision/958575714"" class=""extiw"" title=""en:Special:Redirect/revision/958575714"">Musée Mécanique</a>&quot;",False,5.675667e+07,5.671380e+07,nl.wikipedia.org,/w,https://nl.wikipedia.org,2020-07-19 07:42:35,2020-07-19,Gebruiker:Ymnes/Kladblok5,edit,Ymnes,nlwiki,nl.wikipedia.org,2020-07-19T07:42:35Z,7b3a166c-c161-4df4-b019-77c6f9f67d36,2574275210,0,bb27636c-de78-43dd-960d-24041a78d1bc,mediawiki.recentchange,eqiad.mediawiki.recentchange,https://nl.wikipedia.org/wiki/Gebruiker:Ymnes/Kladblok5
2,None,wiki-changes,2020-07-19 07:42:36.271,/mediawiki/recentchange/1.0.0,True,/* wbsetlabel-add:1|sq */ Cylindroiulus infernalis,1274057862,6974.0,6891.0,False,0,"‎<span dir=""auto""><span class=""autocomment"">Bezeichnung für [sq] hinzugefügt: </span></span> Cylindroiulus infernalis",True,1.234439e+09,1.213659e+09,www.wikidata.org,/w,https://www.wikidata.org,2020-07-19 07:42:35,2020-07-19,Q50810045,edit,SuccuBot,wikidatawiki,www.wikidata.org,2020-07-19T07:42:35Z,3bd89193-8f06-42e8-b2bb-f7d1acd20dc6,2574275211,0,d066edd7-8ff6-4633-9a69-08b76cb380e5,mediawiki.recentchange,eqiad.mediawiki.recentchange,https://www.wikidata.org/wiki/Q50810045
3,None,wiki-changes,2020-07-19 07:42:36.283,/mediawiki/recentchange/1.0.0,False,,417105856,6531.0,6415.0,False,0,,True,1.730532e+08,1.730494e+08,fr.wikipedia.org,/w,https://fr.wikipedia.org,2020-07-19 07:42:35,2020-07-19,Lucio Urtubia,edit,Pierregil83,frwiki,fr.wikipedia.org,2020-07-19T07:42:35Z,3f6831ad-a0c5-47e4-b1cc-bc32b5b5c8cc,2574275212,0,8b6c47b1-4031-406e-8df1-ceb7ad83a5de,mediawiki.recentchange,eqiad.mediawiki.recentchange,https://fr.wikipedia.org/wiki/Lucio_Urtubia
4,None,wiki-changes,2020-07-19 07:42:36.314,/mediawiki/recentchange/1.0.0,True,"/* wbeditentity-update-languages-short:0||nl */ nl-description, [[User:Edoderoobot/Set-nl-description|python code]] - chemische samenstelling",1274057863,2299.0,2220.0,False,0,"‎<span dir=""auto""><span class=""autocomment"">Changed label, description and/or aliases in nl: </span></span> nl-description, <a href=""/wiki/User:Edoderoobot/Set-nl-description"" title=""User:Edoderoobot/Set-nl-description"">python code</a> - chemische samenstelling",True,1.234439e+09,1.100954e+09,www.wikidata.org,/w,https://www.wikidata.org,2020-07-19 07:42:35,2020-07-19,Q83038668,edit,Edoderoobot,wikidatawiki,www.wikidata.org,2020-07-19T07:42:35Z,0c6cd039-5142-48ee-bae0-473292483792,2574275213,0,64bd8595-4a12-426a-ba80-fbc5f343c58b,mediawiki.recentchange,eqiad.mediawiki.recentchange,https://www.wikidata.org/wiki/Q83038668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7399,None,wiki-changes,2020-0

In [11]:
!cd /home/jovyan/work && rm -r data-lake

In [85]:
# Write to data lake parquet partitioned
df_wiki_formatted.write.mode('append').partitionBy("change_timestamp_date", "server_name").parquet("/home/jovyan/work/data-lake/wiki-changes")